# पशु विशेषज्ञ प्रणाली को लागू करना

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) से एक उदाहरण।

इस उदाहरण में, हम एक सरल ज्ञान-आधारित प्रणाली को लागू करेंगे जो कुछ शारीरिक विशेषताओं के आधार पर किसी पशु का निर्धारण करती है। इस प्रणाली को निम्नलिखित AND-OR पेड़ द्वारा दर्शाया जा सकता है (यह पूरे पेड़ का एक हिस्सा है, हम आसानी से इसमें और नियम जोड़ सकते हैं):

![](../../../../lessons/2-Symbolic/images/AND-OR-Tree.png)


## हमारा खुद का एक्सपर्ट सिस्टम शेल बैकवर्ड इंफरेंस के साथ

आइए प्रोडक्शन रूल्स के आधार पर नॉलेज रिप्रेजेंटेशन के लिए एक सरल भाषा को परिभाषित करने की कोशिश करें। हम नियमों को परिभाषित करने के लिए Python क्लासेस का उपयोग करेंगे। मुख्य रूप से 3 प्रकार की क्लासेस होंगी:
* `Ask` एक प्रश्न को दर्शाता है जिसे उपयोगकर्ता से पूछा जाना है। इसमें संभावित उत्तरों का सेट होता है।
* `If` एक नियम को दर्शाता है, और यह केवल नियम की सामग्री को संग्रहीत करने के लिए एक सिंटैक्टिक शॉर्टकट है।
* `AND`/`OR` क्लासेस हैं जो ट्री की AND/OR शाखाओं को दर्शाती हैं। ये केवल अंदर दिए गए तर्कों की सूची को संग्रहीत करती हैं। कोड को सरल बनाने के लिए, सभी कार्यक्षमता पैरेंट क्लास `Content` में परिभाषित की गई है।


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

हमारे सिस्टम में, कार्यशील स्मृति में **तथ्यों** की सूची **गुण-वैल्यू जोड़ों** के रूप में होगी। ज्ञान आधार को एक बड़े शब्दकोश के रूप में परिभाषित किया जा सकता है जो क्रियाओं (नए तथ्य जो कार्यशील स्मृति में डाले जाने चाहिए) को शर्तों से जोड़ता है, जो AND-OR अभिव्यक्तियों के रूप में व्यक्त किए जाते हैं। साथ ही, कुछ तथ्यों को `पूछा` जा सकता है।


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

पिछड़े निष्कर्षण (backward inference) को करने के लिए, हम `Knowledgebase` क्लास को परिभाषित करेंगे। इसमें शामिल होंगे:
* कार्यशील `memory` - एक डिक्शनरी जो गुणों (attributes) को उनके मानों (values) से जोड़ती है।
* Knowledgebase के `rules` - जैसा कि ऊपर परिभाषित किया गया है।

दो मुख्य विधियाँ (methods) हैं:
* `get` - किसी गुण का मान प्राप्त करने के लिए, यदि आवश्यक हो तो निष्कर्षण (inference) करते हुए। उदाहरण के लिए, `get('color')` किसी रंग के स्लॉट का मान प्राप्त करेगा (यह आवश्यक होने पर पूछेगा और कार्यशील मेमोरी में बाद के उपयोग के लिए मान को संग्रहीत करेगा)। यदि हम `get('color:blue')` पूछते हैं, तो यह रंग पूछेगा और फिर `y`/`n` मान लौटाएगा, जो रंग पर निर्भर करेगा।
* `eval` - वास्तविक निष्कर्षण करता है, यानी AND/OR ट्री को पार करता है, उप-लक्ष्यों (sub-goals) का मूल्यांकन करता है, आदि।


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

अब चलिए हमारे पशु ज्ञानकोष को परिभाषित करते हैं और परामर्श करते हैं। ध्यान दें कि यह कॉल आपसे प्रश्न पूछेगा। आप `y`/`n` टाइप करके हां-ना वाले प्रश्नों का उत्तर दे सकते हैं, या लंबे बहुविकल्पीय उत्तरों वाले प्रश्नों के लिए संख्या (0..N) निर्दिष्ट कर सकते हैं।


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## PyKnow का उपयोग करके Forward Inference

अगले उदाहरण में, हम ज्ञान प्रतिनिधित्व के लिए एक लाइब्रेरी [PyKnow](https://github.com/buguroo/pyknow/) का उपयोग करके forward inference को लागू करने की कोशिश करेंगे। **PyKnow** एक लाइब्रेरी है जो Python में forward inference सिस्टम बनाने के लिए डिज़ाइन की गई है, और यह पुराने क्लासिकल सिस्टम [CLIPS](http://www.clipsrules.net/index.html) के समान है।

हम forward chaining को खुद भी बिना ज्यादा समस्या के लागू कर सकते थे, लेकिन साधारण (naive) कार्यान्वयन आमतौर पर बहुत प्रभावी नहीं होते। अधिक प्रभावी नियम मिलान (rule matching) के लिए एक विशेष एल्गोरिदम [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) का उपयोग किया जाता है।


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

हम अपने सिस्टम को `KnowledgeEngine` का सबक्लास बनाकर एक क्लास के रूप में परिभाषित करेंगे। प्रत्येक नियम को `@Rule` एनोटेशन के साथ एक अलग फ़ंक्शन द्वारा परिभाषित किया जाता है, जो यह निर्दिष्ट करता है कि नियम कब सक्रिय होना चाहिए। नियम के अंदर, हम `declare` फ़ंक्शन का उपयोग करके नए तथ्य जोड़ सकते हैं, और उन तथ्यों को जोड़ने से आगे की अनुमान इंजन द्वारा कुछ और नियमों को कॉल किया जाएगा।


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

एक बार जब हम एक ज्ञान आधार परिभाषित कर लेते हैं, तो हम अपनी कार्यशील स्मृति को कुछ प्रारंभिक तथ्यों से भरते हैं, और फिर `run()` विधि को कॉल करते हैं ताकि निष्कर्षण किया जा सके। आप परिणामस्वरूप देख सकते हैं कि नए निष्कर्षित तथ्य कार्यशील स्मृति में जोड़े जाते हैं, जिसमें जानवर के बारे में अंतिम तथ्य भी शामिल है (यदि हमने सभी प्रारंभिक तथ्यों को सही तरीके से सेट किया है)।


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
